# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

D:\Git\Recommendation-Systems\src\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5000


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=200)

[863655,
 844179,
 824738,
 871756,
 872062,
 845208,
 854852,
 831390,
 854976,
 843114,
 854270,
 872791,
 844165,
 871352,
 827683,
 859075,
 819112,
 865456,
 870547,
 819765,
 828106,
 828370,
 820122,
 819982,
 833025,
 866211,
 855914,
 854261,
 823990,
 845109,
 830795,
 869047,
 870195,
 871094,
 820551,
 871208,
 846823,
 856827,
 825218,
 871876,
 865357,
 855350,
 831509,
 834303,
 843343,
 824176,
 837751,
 835595,
 821675,
 828278,
 819518,
 864312,
 821930,
 845705,
 858918,
 823255,
 850801,
 865528,
 822161,
 852851,
 858418,
 832678,
 825343,
 855626,
 830015,
 859154,
 853221,
 847790,
 850281,
 863447,
 831728,
 842455,
 868423,
 857006,
 838563,
 870428,
 872186,
 824546,
 871611,
 864279,
 835476,
 824759,
 861282,
 825539,
 842817,
 863208,
 870933,
 855279,
 858424,
 853038,
 857939,
 845675,
 853354,
 838940,
 829550,
 825006,
 869895,
 868909,
 835300,
 843441,
 843567,
 825118,
 829456,
 838396,
 824915,
 871470,
 842342,
 856284,
 848029,
 823570,
 821219,
 

In [6]:
recommender.get_own_recommendations(2375, N=200)

[859075,
 844165,
 844179,
 854852,
 845208,
 862349,
 866211,
 847066,
 835300,
 847962,
 825541,
 866871,
 857006,
 850102,
 840640,
 857503,
 823704,
 865178,
 863447,
 871756,
 825994,
 861445,
 854852,
 862349,
 844179,
 866211,
 859075,
 844165,
 845208,
 833025,
 847982,
 824005,
 871756,
 865178,
 832678,
 834117,
 868764,
 870547,
 860975,
 856772,
 857503,
 835098,
 861445,
 825541,
 854405,
 823704,
 870515,
 839419,
 863447,
 819978,
 865456,
 872137,
 821083,
 866871,
 859010,
 864774,
 863802,
 838186,
 852856,
 845307,
 828867,
 845078,
 852486,
 867188,
 849202,
 831628,
 860299,
 865528,
 858302,
 844818,
 845193,
 823990,
 834103,
 861279,
 827683,
 857006,
 819765,
 865174,
 819927,
 848071,
 871611,
 850102,
 856252,
 828106,
 845705,
 823721,
 837270,
 830887,
 865705,
 848029,
 866755,
 862732,
 827180,
 822785,
 826385,
 843756,
 842140,
 825994,
 825343,
 869577,
 871061,
 846823,
 859427,
 848319,
 871440,
 819255,
 854754,
 827656,
 840601,
 856827,
 825365,
 

In [7]:
recommender.get_similar_items_recommendation(2375, N=200)

[833025,
 845208,
 829456,
 861821,
 861821,
 844179,
 823570,
 858918,
 858302,
 871756,
 870547,
 871611,
 871611,
 828106,
 823990,
 834826,
 855747,
 845208,
 865456,
 828106,
 870515,
 843346,
 836608,
 864774,
 855138,
 834826,
 827570,
 822860,
 864774,
 870547,
 822953,
 825006,
 825170,
 834631,
 870515,
 851913,
 844818,
 863885,
 825343,
 834826,
 859075,
 854852,
 862349,
 844179,
 866211,
 859075,
 844165,
 845208,
 833025,
 847982,
 824005,
 871756,
 865178,
 832678,
 834117,
 868764,
 870547,
 860975,
 856772,
 857503,
 835098,
 861445,
 825541,
 854405,
 823704,
 870515,
 839419,
 863447,
 819978,
 865456,
 872137,
 821083,
 866871,
 859010,
 864774,
 863802,
 838186,
 852856,
 845307,
 828867,
 845078,
 852486,
 867188,
 849202,
 831628,
 860299,
 865528,
 858302,
 844818,
 845193,
 823990,
 834103,
 861279,
 827683,
 857006,
 819765,
 865174,
 819927,
 848071,
 871611,
 850102,
 856252,
 828106,
 845705,
 823721,
 837270,
 830887,
 865705,
 848029,
 866755,
 862732,
 

In [8]:
recommender.get_similar_users_recommendation(2375, N=200)

[839656,
 823990,
 854270,
 847982,
 871337,
 825539,
 849870,
 831063,
 834491,
 855626,
 819112,
 834117,
 831628,
 825343,
 824813,
 833025,
 831628,
 871754,
 838186,
 845705,
 842140,
 819765,
 819308,
 821890,
 826099,
 847790,
 850281,
 853221,
 859191,
 870195,
 871876,
 822785,
 822346,
 866292,
 861792,
 825970,
 860439,
 846764,
 861282,
 870735,
 870515,
 864774,
 863885,
 858514,
 856772,
 852486,
 847720,
 843171,
 841025,
 839747,
 833193,
 831557,
 830156,
 828106,
 828727,
 826784,
 825343,
 825006,
 829972,
 827667,
 824303,
 824180,
 823990,
 819594,
 828106,
 831063,
 835431,
 855916,
 871440,
 869577,
 866878,
 865456,
 861282,
 860975,
 854405,
 854373,
 854261,
 851324,
 850139,
 848029,
 845675,
 839419,
 836357,
 835530,
 837865,
 835619,
 834303,
 832513,
 831628,
 830935,
 828867,
 828588,
 828106,
 827047,
 826860,
 825343,
 823990,
 823176,
 822944,
 822140,
 821565,
 821344,
 839262,
 840601,
 841282,
 852486,
 845307,
 841236,
 838010,
 837963,
 835476,
 

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
# Для чистоты эксперимента удалим новых юзеров

result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(data_train_lvl_1['user_id'])]

result_lvl_1['als_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1['similar_items_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
result_lvl_1['similar_users_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1['own_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1['top_rec'] = [recommender.overall_top_purchases[:200] for x in np.arange(len(result_lvl_1))]
result_lvl_1.head(2)

,user_id,actual,als_rec,similar_items_rec,similar_users_rec,own_rec,top_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 852662, 839766, 859589, 840227, 85352...","[865558, 833988, 848832, 854405, 867188, 82370...","[854754, 870515, 822049, 823775, 839766, 84682...","[856942, 854852, 862349, 844179, 847982, 86545...","[854852, 862349, 844179, 866211, 859075, 84416..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[845675, 822812, 852486, 837664, 821324, 85323...","[850102, 820222, 849202, 40885, 259120, 279994...","[854852, 844179, 862349, 865705, 820301, 83359...","[854852, 862349, 844179, 866211, 859075, 84416..."


In [11]:
recall_als = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec'], row['actual'], k=200), axis=1).mean()
recall_similar_items = result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_rec'],
                                                                  row['actual'], k=200), axis=1).mean()
recall_similar_users = result_lvl_1.apply(lambda row: recall_at_k(row['similar_users_rec'],
                                                                  row['actual'], k=200), axis=1).mean()
recall_own = result_lvl_1.apply(lambda row: recall_at_k(row['own_rec'], row['actual'], k=200), axis=1).mean()
recall_top = result_lvl_1.apply(lambda row: recall_at_k(row['top_rec'], row['actual'], k=200), axis=1).mean()\

print('''
Recall@200:

als_rec = {0},
similar_items_rec = {1},
similar_users_rec = {2},
own_rec = {3},
top_rec = {4}
'''.format(recall_als, recall_similar_items, recall_similar_users, 
           recall_own, recall_top))


Recall@200:

als_rec = 0.03754506147699352,
similar_items_rec = 0.04271838913209686,
similar_users_rec = 0.01673712155630251,
own_rec = 0.04709253716628431,
top_rec = 0.04378959727277339



Видим что own recommendations и top-popular дают лучший recall.  
Далее посмотрим на различные k для ALS.

In [12]:
result_lvl_1['als_rec20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=20))
result_lvl_1['als_rec50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1['als_rec100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=100))
result_lvl_1['als_rec500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=500))
result_lvl_1['als_rec300'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=300))


result_lvl_1.head(2)

,user_id,actual,als_rec,similar_items_rec,similar_users_rec,own_rec,top_rec,als_rec20,als_rec50,als_rec100,als_rec500,als_rec300
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 852662, 839766, 859589, 840227, 85352...","[865558, 833988, 848832, 854405, 867188, 82370...","[854754, 870515, 822049, 823775, 839766, 84682...","[856942, 854852, 862349, 844179, 847982, 86545...","[854852, 862349, 844179, 866211, 859075, 84416...","[856942, 852662, 839766, 859589, 840227, 85352...","[856942, 852662, 839766, 859589, 840227, 85352...","[856942, 852662, 839766, 859589, 840227, 85352...","[856942, 852662, 839766, 859589, 840227, 85352...","[856942, 852662, 839766, 859589, 840227, 85352..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[845675, 822812, 852486, 837664, 821324, 85323...","[850102, 820222, 849202, 40885, 259120, 279994...","[854852, 844179, 862349, 865705, 820301, 83359...","[854852, 862349, 844179, 866211, 859075, 84416...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[279994, 40885, 70302, 32916, 854852, 259120, ...","[279994, 40885, 70302, 32916, 854852, 259120, ..."


In [13]:
recall_als200 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec'], row['actual'], k=200), axis=1).mean()
recall_als20 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec20'], row['actual'], k=20), axis=1).mean()
recall_als50 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec50'], row['actual'], k=50), axis=1).mean()
recall_als100 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec100'], row['actual'], k=100), axis=1).mean()
recall_als300 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec300'], row['actual'], k=300), axis=1).mean()
recall_als500 = result_lvl_1.apply(lambda row: recall_at_k(row['als_rec500'], row['actual'], k=500), axis=1).mean()

print('''
ALS Recall@200 = {0},
ALS Recall@20 = {1},
ALS Recall@50 = {2},
ALS Recall@100 = {3},
ALS Recall@300 = {4},
ALS Recall@500 = {5}
'''.format(recall_als200, recall_als20, recall_als50, 
           recall_als100, recall_als300, recall_als500))


ALS Recall@200 = 0.03754506147699352,
ALS Recall@20 = 0.013719186319619353,
ALS Recall@50 = 0.020430030218792795,
ALS Recall@100 = 0.027502722763392107,
ALS Recall@300 = 0.04407905428862567,
ALS Recall@500 = 0.054717679368883915



Разумным значением k будет являться 200-300, далее идет естественный затухающий рост, но брать больше нет смысла, т.к. recall увеличивается незначительно, а эффективность\скорость модели второго уровня будет падать от увеличения кол-ва товаров.

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [14]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
users_lvl_2.head(3)

,user_id,candidates
0,2070,"[844179, 859075, 866211, 832678, 823704, 86615..."
1,2021,"[844165, 835578, 846823, 871756, 844179, 84356..."
2,1753,"[862981, 836924, 843339, 853998, 835634, 82048..."


In [15]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1  # фиктивная пересенная

users_lvl_2.head()

,user_id,item_id,drop
0,2070,844179,1
0,2070,859075,1
0,2070,866211,1
0,2070,832678,1
0,2070,823704,1


In [16]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)
targets_lvl_2.head()

,user_id,item_id,target
0,2070,844179,0.0
1,2070,859075,1.0
2,2070,866211,0.0
3,2070,832678,0.0
4,2070,823704,0.0


Добавляем фичи. Начнем с фичей пользователей - средний чек и средняя сумма покупки в категории. Добавляем фичи товаров - Цена и средняя цена товара в категории

In [17]:
user_purchases = data_train_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
user_mean_bill = user_purchases.groupby('user_id')['sales_value'].mean().reset_index()
user_mean_bill.rename(columns={'sales_value': 'mean_bill'}, inplace=True)

data = data_train_lvl_2.merge(item_features, on='item_id', how='left')

user_mean_dep_purchase = data.groupby(['user_id', 'sub_commodity_desc'], sort=False)['sales_value'].mean().reset_index()
user_mean_dep_purchase.rename(columns={'sales_value': 'mean_dep_purchase'}, inplace=True)

In [18]:
targets_lvl_2 = targets_lvl_2.merge(user_mean_bill, on='user_id', how='left')
targets_lvl_2.head()

,user_id,item_id,target,mean_bill
0,2070,844179,0.0,14.355581
1,2070,859075,1.0,14.355581
2,2070,866211,0.0,14.355581
3,2070,832678,0.0,14.355581
4,2070,823704,0.0,14.355581


In [19]:
data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
items_price = data.groupby(['item_id'])['price'].mean().reset_index()
dep_mean_item_price = data.groupby(['sub_commodity_desc'])['price'].mean().reset_index()
dep_mean_item_price.rename(columns={'price': 'mean_dep_price'}, inplace=True)

In [20]:
items_deps = data[['item_id', 'sub_commodity_desc']].copy().drop_duplicates()
targets_lvl_2 = targets_lvl_2.merge(items_deps, on='item_id', how='left')

In [21]:
targets_lvl_2 = targets_lvl_2.merge(items_price, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(dep_mean_item_price, on='sub_commodity_desc', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_mean_dep_purchase, on=['user_id', 'sub_commodity_desc'], how='left')
targets_lvl_2.drop('sub_commodity_desc', axis=1, inplace=True)
targets_lvl_2= targets_lvl_2.fillna(0)
targets_lvl_2.head()

,user_id,item_id,target,mean_bill,price,mean_dep_price,mean_dep_purchase
0,2070,844179,0.0,14.355581,3.737419,3.892002,10.450000
1,2070,859075,1.0,14.355581,1.575195,1.917284,2.673333
2,2070,866211,0.0,14.355581,4.335975,4.379384,0.000000
3,2070,832678,0.0,14.355581,2.416429,2.155000,0.000000
4,2070,823704,0.0,14.355581,3.102000,2.577399,0.000000


Подаем данные в модель

In [22]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [23]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7)
lgb.fit(X_train, y_train)

C:\Users\horne\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\horne\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 1.69 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [24]:
train_preds = lgb.predict_proba(X_train)

train_preds = pd.DataFrame(train_preds, columns=['0', '1'])
train_preds.head()

,0,1
0,0.511502,0.488498
1,0.737692,0.262308
2,0.999996,0.000004
3,0.999999,0.000001
4,0.999999,0.000001


In [25]:
targets_lvl_2['target_pred'] = train_preds['1']
targets_lvl_2 = targets_lvl_2.sort_values(['user_id', 'target_pred'], ascending=[False, False])
targets_lvl_2


,user_id,item_id,target,mean_bill,price,mean_dep_price,mean_dep_purchase,target_pred
275399,2500,859237,1.0,41.700000,1.285652,1.180357,1.29,3.041720e-01
275538,2500,843756,0.0,41.700000,0.988494,1.300899,1.50,2.989630e-01
275500,2500,826842,0.0,41.700000,2.190000,2.869855,1.99,1.790013e-01
275369,2500,839035,0.0,41.700000,3.990000,3.562941,4.98,1.671717e-01
275452,2500,856455,1.0,41.700000,2.495000,3.308350,2.50,1.645783e-01
...,...,...,...,...,...,...,...,...
176074,1,870269,0.0,48.825714,2.390000,1.803373,0.00,9.423589e-07
176113,1,871722,0.0,48.825714,2.445714,2.493443,0.00,9.140886e-07
176095,1,869755,0.0,48.825714,2.526667,1.858049,0.00,1.822034e-07
176054,1,869874,0.0,48.825714,11.040000,4.667468,0.00,1.070515e-12


Результат предсказания модели второго уровня сохраняем в preds_lgbm

In [26]:
preds_lgbm = targets_lvl_2[['user_id', 'item_id']].copy().drop_duplicates()
preds_lgbm = preds_lgbm.groupby('user_id')['item_id'].apply(list).reset_index()
preds_lgbm

,user_id,item_id
0,1,"[872137, 862349, 867188, 856942, 832990, 84876..."
1,2,"[866211, 833025, 844179, 859075, 868015, 28461..."
2,4,"[840322, 849870, 866025, 827667, 831063, 82609..."
3,6,"[845208, 825541, 858424, 853235, 834303, 83530..."
4,7,"[859075, 870424, 853643, 855557, 866172, 86234..."
...,...,...
2130,2496,"[865456, 844179, 824915, 859075, 830156, 86718..."
2131,2497,"[845208, 864774, 844175, 859075, 859010, 84798..."
2132,2498,"[859075, 863802, 860804, 844991, 822073, 83733..."
2133,2499,"[859075, 845193, 834103, 854042, 846830, 85167..."


Сравниваем precision@5 модели 1го уровня и двухуровневой модели на data_val_lvl_2

In [27]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.rename(columns={'item_id': 'actual'}, inplace=True)
result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(train_users)]

result_lvl_2['als_1'] = result_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_2 = result_lvl_2.merge(preds_lgbm, on='user_id', how='left')
result_lvl_2.rename(columns={'item_id': 'lgbm_2'}, inplace=True)

result_lvl_2.head(2)

,user_id,actual,als_1,lgbm_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 852662, 839766, 859589, 840227, 85352...","[872137, 862349, 867188, 856942, 832990, 84876..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[871756, 844179, 844165, 859075, 846823, 87054...",NaN


Возникла ситуация, когда некоторые юзеры есть в первой части, отсуствуют во второй и снова появляются в третьей. Для таких модель второго уровня не выдает рекомендаций, и для замеров метрики придется их удалить.

In [28]:
result_lvl_2 = result_lvl_2[result_lvl_2['lgbm_2'].notna()]
result_lvl_2.head()

,user_id,actual,als_1,lgbm_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 852662, 839766, 859589, 840227, 85352...","[872137, 862349, 867188, 856942, 832990, 84876..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[849618, 867469, 866776, 831161, 827614, 85024...","[845208, 825541, 858424, 853235, 834303, 83530..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[849032, 866211, 846634, 857554, 854965, 86657...","[859075, 870424, 853643, 855557, 866172, 86234..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[866211, 850834, 826578, 870882, 840938, 85722...","[844179, 867188, 858918, 859075, 861706, 84925..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[832810, 864658, 843911, 836163, 871722, 83359...","[832526, 865501, 862799, 868131, 842707, 83710..."


In [29]:
precision5_als_1 = result_lvl_2.apply(lambda row: precision_at_k(row['als_1'], row['actual']), axis=1).mean()
precision5_lgmb_2 = result_lvl_2.apply(lambda row: precision_at_k(row['lgbm_2'], row['actual']), axis=1).mean()


print('''
One-level Precision@5 = {0},
Two-Level Precision@5 = {1},

'''.format(precision5_als_1, precision5_lgmb_2))


One-level Precision@5 = 0.0698896479243303,
Two-Level Precision@5 = 0.0953231739358905,




У двухуровневой модели Precision@5 выше примерно на 2,5%

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)